# Data Cleaning & Preparation For Text Summarization

Text summarization has an easy to understand input and output. The input is a body of text, and the ouptut is a signficantally smaller summary. To clean the data we can perform the following operations. 

* Eliminate rows with null content or summary
* Eliminate rows where the body or summary is too long or too short.
* Remove unusual characters
* Lower case the text
* Remove rare tokens

After the modeling is started this list may need to be expanded upon and the script re-run. 

The table will look like this:


| content                             | summary                            |
|-------------------------------------|------------------------------------|
| Content between 100 and 1000 tokens | Sumamry between  20 and 100 tokens |



The information on the individual data sources is in the SummarizationDataSources.ipynb notebook.






In [1]:
from pyspark.sql import functions as F
from pyspark.sql import udf
import contractions
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /home/jzeimen/dev/ecosystem/spark/spark-tensorflow-connector/target/spark-tensorflow-connector_2.12-1.11.0.jar pyspark-shell'
from pyspark.sql.functions import lower, col
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Summarization") \
        .master("local[*]") \
        .config("spark.jars", "/home/jzeimen/dev/ecosystem/spark/spark-tensorflow-connector/target/spark-tensorflow-connector_2.12-1.11.0.jar")\
        .config("spark.driver.memory", "25g")\
        .config("spark.driver.maxResultSize", "8g")\
        .getOrCreate()

In [2]:
# What characters are we dealing with?
reddit_sdf = spark.read.format("tfrecords").load("/home/jzeimen/tensorflow_datasets/reddit/1.0.0/reddit*")
chars_sdf = reddit_sdf.withColumn("chars", F.split(reddit_sdf.content, ""))
chars_pdf = chars_sdf.select(F.explode('chars')).withColumnRenamed("col","chars").groupBy("chars").count().toPandas()
print(f"Found {chars_pdf.shape[0]} different characters, here are the top 200")
print(chars_pdf.sort_values("count", ascending=False).chars.values[:200])

Found 6376 different characters, here are the top 200
[' ' 'e' 't' 'a' 'o' 'n' 'i' 's' 'r' 'h' 'l' 'd' 'u' 'm' 'c' 'g' 'y' 'w'
 'f' 'p' 'b' '.' 'I' ',' 'v' 'k' "'" '\n' 'T' 'S' 'A' 'x' 'j' '"' '(' ')'
 '-' 'W' 'H' '0' 'M' 'B' '1' 'O' 'E' 'C' '' 'N' '?' 'D' '2' 'P' 'z' 'F'
 'R' 'L' 'q' '/' ':' 'G' '3' '5' 'Y' '!' '4' 'U' 'J' '6' 'K' 'V' '’' '8'
 '9' '7' ']' '[' ';' '>' '$' '%' 'X' '+' '&' 'Q' 'Z' '=' '”' '“' '*' '^'
 '~' '_' '#' 'é' '…' '–' '|' '@' 'ä' '<' '—' 'å' '‘' '£' 'ö' '\\' '•' 'α'
 '{' '}' 'í' 'á' 'ο' 'ó' '´' 'ι' 'ε' 'ü' 'τ' '°' 'ν' '€' 'о' 'ã' 'ă' 'à'
 'а' 'ç' 'è' 'е' 'π' 'и' 'σ' 'ρ' 'μ' 'κ' 'н' 'т' 'υ' '͡' 'λ' 'ς' 'ø' 'æ'
 '`' 'ñ' 'η' 'с' '͜' 'ê' 'р' 'ل' '·' 'в' 'ú' 'ί' 'ά' 'ό' 'л' 'к' 'ð' 'š'
 'д' 'ຈ' 'γ' 'ಠ' 'î' 'έ' 'м' 'δ' 'ß' 'ω' 'ا' 'ʖ' 'у' 'п' 'â' 'ș' '̶' 'č'
 'θ' 'ï' '\xad' '\ufeff' 'χ' '█' '™' '×' 'ć' '༼' '༽' 'я' 'َ' 'ή' 'º' 'ț'
 'い' '─' 'б' '§']


Based on this info, let's get rid of anything that isn't a letter or a number. Then we can also normalize the sentence endings so they are just periods. 

We can also normalize contractions to make things easier. For example, "can't" would be come "can not". From the data set analysis there is a wide range in lengths. For now lets make sure our summaries are between 20 and 100 tokens long, the content is between 100 and 1000 tokens long. We can also make sure that the summary is shorter than the actual content. 

In [19]:
def replace_contractions(x):
    try:
        return contractions.fix(x)
    except:
        return None

fix_contractions = F.udf(lambda x: replace_contractions(x))

def clean_column(sdf, input_col, output_col):
    # Drop anything with null values, these rows are not helpful at all
    out_sdf = sdf.na.drop()
    # Rename the columns
    out_sdf = out_sdf.withColumn(output_col, fix_contractions(out_sdf[input_col]))
    out_sdf = out_sdf.na.drop()

    # We'll just keep periods for sentence endings
    out_sdf = out_sdf.withColumn(output_col, F.regexp_replace(output_col, r"[!?]", "."))
    # Just Keep letters and numbers periods and commas
    out_sdf = out_sdf.withColumn(output_col, F.regexp_replace(output_col, r"[^A-Za-z0-9.,]+", " "))
    # Change everything to lower case
    out_sdf = out_sdf.withColumn(output_col, F.lower(out_sdf[output_col]))
    out_sdf = out_sdf.na.drop()
    return out_sdf
    
def add_token_counts(sdf, col):
    out_sdf = sdf.withColumn(col + "_token_count", F.size(F.split(sdf[col], R"\W"))) 
    return out_sdf
    
def clean_df(sdf, content_col, summary_col):
    final_sdf = clean_column(sdf, content_col, content_col)
    final_sdf = clean_column(final_sdf, summary_col, summary_col)
    final_sdf = final_sdf.withColumnRenamed(content_col, "content")
    final_sdf = final_sdf.withColumnRenamed(summary_col, "summary")
    final_sdf = add_token_counts(final_sdf, "content")
    final_sdf = add_token_counts(final_sdf, "summary")
    final_sdf = final_sdf.where("100 < content_token_count AND content_token_count < 1000")
    final_sdf = final_sdf.where("20 < summary_token_count AND summary_token_count < 100")
    final_sdf = final_sdf.where("content_token_count > summary_token_count")
    return final_sdf

In [4]:
%%time
print("Working on cnn/dailymail dataset...")
news_sdf = spark.read.format("tfrecords").load("/home/jzeimen/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0/cnn*")
print(f"Count before cleaning {news_sdf.count()}")
news_sdf = clean_df(news_sdf, "article", "highlights").cache()
news_sdf.write.format("tfrecords").save("clean/news")
print(f"Count after cleaning {news_sdf.count()}")

Working on cnn/dailymail dataset...
Count before cleaning 311971
Count after cleaning 223763
CPU times: user 30.2 ms, sys: 12 ms, total: 42.2 ms
Wall time: 3min 54s


In [14]:
%%time
print("Working on reddit dataset...")
reddit_sdf = spark.read.format("tfrecords").load("/home/jzeimen/tensorflow_datasets/reddit/1.0.0/reddit*").cache()

Working on reddit dataset...
CPU times: user 2 ms, sys: 7.96 ms, total: 9.96 ms
Wall time: 1min 1s


In [21]:
%%time

print(f"Count before cleaning {reddit_sdf.count()}")
reddit_sdf = clean_df(reddit_sdf, "content", "summary").cache()
reddit_sdf = reddit_sdf.select("content", "summary", "content_token_count", "summary_token_count")
#reddit_sdf.write.format("tfrecords").save("clean/reddit")
print(f"Count after cleaning {reddit_sdf.count()}")

Count before cleaning 3848330
Count after cleaning 1584283
CPU times: user 126 ms, sys: 49.7 ms, total: 175 ms
Wall time: 21min 9s


In [6]:
%%time
print("Working on wikihow dataset...")
wiki_sdf = spark.read.csv("WikiHow-Dataset/wikihowAll.csv", header=True, multiLine=True, escape="\"").na.drop()
print(f"Count before cleaning {wiki_sdf.count()}")
wiki_sdf = clean_df(wiki_sdf, "text", "headline").cache()
wiki_sdf.write.format("tfrecords").save("clean/wikihow")
print(f"Count after cleaning {wiki_sdf.count()}")

Working on wikihow dataset...
Count before cleaning 214294
Count after cleaning 115251
CPU times: user 19.3 ms, sys: 14.2 ms, total: 33.6 ms
Wall time: 3min 5s


In [8]:
wiki_sdf.count()

115251

Here's some examples after cleaning

In [19]:
reddit_clean = spark.read.format("tfrecords").load("/home/jzeimen/Dropbox/springboard/springboard-capstone/data/clean/reddit/par*")

In [26]:
pd.set_option("display.max_colwidth", None)
reddit_clean.limit(100).toPandas()[20:22][["content", "summary"]]

,content,summary
20,"football is the main driver of college sports these days. i do not like that fact as i am not a big cfb fan but i am not stupid enough to deny what is in front of me. i am also one who think the best conferences fit geographically. that being said, these are the ones i think are better and worse from a football standpoint. best any team that moved to a bcs conference and did not miss a beat. i would say atm and nebraska are the only two good moves right now. wvu got exposed for their lack of defense. missouri is in misery. colorado is terrible and utah got exposed for not feasting on mountain west teams year in and year out. worst every team in the big east conference. for multiple reasons. it is the only major conference to lose more than 3 teams to other conferences in the bcs era. it lost 6, 7 if you count tcu which never played a big east down. it has the perception bestowed on it by national media as being undeserving of major conference status despite it is bowl record since 2006 when the new teams came in. the acc has a worse bcs bowl record than the big east and i believe even has a worse bowl record than the big east overall since that time. if it is not worse, it is certainly close. since the big east is being perceived as being weak, the top teams in the big east louisville, cincinnati, rutgers are automatically seen as weak despite that not necessarily being true. yes, i know that mac teams have taken out uc and ru this year. mac teams have also taken out some b1g teams as well, but those teams are not being seen as being weak. boise and sdsu joined up for bcs exposure. in reality neither team should be in a conference based in rhode island and that exposure they wanted through the big east goes away in 2014 when the big east seen as being a weak little sister gets no automatic bid to the premier bowls the sec, b12, b1g, pac 12, and acc which should not be automatically given a seat statistically they do not deserve if the big east does not deserve a seat are getting. houston, smu, memphis, temple, and ucf have the same problem. plus navy is getting in in a few years time, and although it does not matter to them, they will not get any bcs bowl exposure at all. notre dame was in the conference for everything but football but never joined the big east in football. what i find very insulting is that they allegedly agreed to play 3 big east teams but they were typically traditional rivals like pitt, syracuse and then rutgers, uconn or south florida, the latter in 2 year deals. i know usf was a one shot deal but did not and then did not play wvu, uc, or uofl where cincinnati and louisville have significant catholic populations. now notre dame has agreed to play 5 acc teams a year. that seems like a gigantic middle finger to the conference that agreed to take their non football teams out of the mid continent conference to a larger stage for 15 years. the media does not deem the big east relevant and the big east has certainly obliged that decision by not shoring up football after boston college and miami wanted to leave vt i think got thrusted on the acc and added additional basketball schools for whatever reason. truth be told, i want out. i really do not care if the big east would be an easy conference to win. big east decision making essentially damned my school back to the perception of mediocrity in football and we look to always be on the outside looking in. give me the big 12. west virginia will be happy and if our football coach leaves, it will not be because of the glass ceiling being in the big east has not placed on us. basketball we are good anywhere.","winners atm, nebraska losers current and future big east, plus fuck the acc, syracuse and pitt, get louisville out before being in the big east kills louisville football."
21,"my mom had me when she was 16, nearly 17. my dad was nearly 18 at the time. i was born a month premature, 8 days before her birthday, 3 days before his. life was alr